# Sparse Matrices 

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy import sparse
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sns.set_context('notebook', font_scale=1.5)

## A review of the essentials

There are many applications in which we deal with matrices that are mostly zeros. For example, a matrix representing a text corpus is very sparse - there are many thousands of words used but each document only uses a small subset of the total. Social networks are similar because there are a lot of people, but most people are only connected to a few hundred or thousand others directly. Storing such a social network as a sparse rather than dense matrix  will offer orders of magnitude reductions in memory requirements and corresponding speed-ups in computation.

### Coordinate format

The simplest sparse  matrix format is built from the coordinates and values of the non-zero entries.

#### From dense matrix

In [4]:
A = np.random.poisson(0.2, (5,15)) * np.random.randint(0, 10, (5, 15))
A

array([[0, 0, 0, 0, 0, 0, 6, 0, 0, 8, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 8, 4, 0, 0],
       [0, 7, 0, 0, 0, 0, 8, 0, 0, 0, 2, 0, 4, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 9, 0, 0],
       [4, 0, 4, 0, 3, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [7]:
rows, cols = np.nonzero(A)
vals = A[rows, cols]

In [8]:
vals

array([6, 8, 4, 8, 4, 7, 8, 2, 4, 7, 9, 4, 4, 3, 7])

In [9]:
rows

array([0, 0, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4, 4, 4, 4])

In [10]:
cols

array([ 6,  9,  9, 11, 12,  1,  6, 10, 12,  9, 12,  0,  2,  4,  5])

In [11]:
X1 = sparse.coo_matrix(A)
X1

<5x15 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in COOrdinate format>

In [12]:
print(X1)

  (0, 6)	6
  (0, 9)	8
  (1, 9)	4
  (1, 11)	8
  (1, 12)	4
  (2, 1)	7
  (2, 6)	8
  (2, 10)	2
  (2, 12)	4
  (3, 9)	7
  (3, 12)	9
  (4, 0)	4
  (4, 2)	4
  (4, 4)	3
  (4, 5)	7


#### From coordinates

Note that the (values, (rows, cols)) argument is a single tuple.

In [13]:
X2 = sparse.coo_matrix((vals, (rows, cols)))
X2

<5x13 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in COOrdinate format>

In [14]:
print(X2)

  (0, 6)	6
  (0, 9)	8
  (1, 9)	4
  (1, 11)	8
  (1, 12)	4
  (2, 1)	7
  (2, 6)	8
  (2, 10)	2
  (2, 12)	4
  (3, 9)	7
  (3, 12)	9
  (4, 0)	4
  (4, 2)	4
  (4, 4)	3
  (4, 5)	7


#### Convert back to dense matrix

In [15]:
X2.todense()

matrix([[0, 0, 0, 0, 0, 0, 6, 0, 0, 8, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 8, 4],
        [0, 7, 0, 0, 0, 0, 8, 0, 0, 0, 2, 0, 4],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 9],
        [4, 0, 4, 0, 3, 7, 0, 0, 0, 0, 0, 0, 0]])

## Compressed Sparse Row and Column formats

When we have repeated entries in the rows or cols, we can remove the redundancy by indicating the  location of the  first occurrence of a value and its increment instead of the full coordinates. Note that the final index location  must be the number of rows or cols since there is no other way to know the shape. These are known as CSR or CSC formats.

In [16]:
np.vstack([rows, cols])

array([[ 0,  0,  1,  1,  1,  2,  2,  2,  2,  3,  3,  4,  4,  4,  4],
       [ 6,  9,  9, 11, 12,  1,  6, 10, 12,  9, 12,  0,  2,  4,  5]])

In [17]:
indptr = np.r_[np.searchsorted(rows, np.unique(rows)), len(rows)]
indptr

array([ 0,  2,  5,  9, 11, 15])

In [18]:
X3 = sparse.csr_matrix((vals, cols, indptr))
X3

<5x13 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [19]:
print(X3)

  (0, 6)	6
  (0, 9)	8
  (1, 9)	4
  (1, 11)	8
  (1, 12)	4
  (2, 1)	7
  (2, 6)	8
  (2, 10)	2
  (2, 12)	4
  (3, 9)	7
  (3, 12)	9
  (4, 0)	4
  (4, 2)	4
  (4, 4)	3
  (4, 5)	7


In [20]:
X3.todense()

matrix([[0, 0, 0, 0, 0, 0, 6, 0, 0, 8, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 8, 4],
        [0, 7, 0, 0, 0, 0, 8, 0, 0, 0, 2, 0, 4],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 9],
        [4, 0, 4, 0, 3, 7, 0, 0, 0, 0, 0, 0, 0]])

#### Casting from COO format

Because the coordinate format is more intuitive, it is often more convenient to first create a COO matrix then cast to CSR or CSC form.

In [21]:
X4 = X2.tocsr()

In [22]:
X4

<5x13 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [23]:
rows = np.repeat([0,1], 4)
cols = np.repeat([0,1], 4)
vals = np.arange(8)

In [24]:
rows

array([0, 0, 0, 0, 1, 1, 1, 1])

In [25]:
cols

array([0, 0, 0, 0, 1, 1, 1, 1])

In [26]:
vals

array([0, 1, 2, 3, 4, 5, 6, 7])

In [27]:
X5 = sparse.coo_matrix((vals, (rows, cols)))

In [28]:
X5.todense()

matrix([[ 6,  0],
        [ 0, 22]])

## Question 1: Preparing the data

Read in the data in from `airports.csv` and `names.csv`.  We are interested in the following columns

* ORIGIN_AIRPORT_ID	
* DEST_AIRPORT_ID

The second file will help us figure out what each code means

Read both into data frames in a way that will allow you to lookup the `description` of an airport based on its ID.

In [157]:
## ANSWER
data = pd.read_csv('./data/airports.csv', sep=',')
data = airports.drop(columns='Unnamed: 2')

names = pd.read_csv('./data/names.csv', sep=',')
names_dict = names.set_index('Code')['Description'].to_dict()



,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID
0,10135,10397
1,10135,10397
2,10135,10397
3,10135,10397
4,10135,10397
...,...,...
445822,16218,14107
445823,16218,14107
445824,16218,14107
445825,16218,14107


In [111]:
names

,Code,Description
0,10001,"Afognak Lake, AK: Afognak Lake Airport"
1,10003,"Granite Mountain, AK: Bear Creek Mining Strip"
2,10004,"Lik, AK: Lik Mining Camp"
3,10005,"Little Squaw, AK: Little Squaw Airport"
4,10006,"Kizhuyak, AK: Kizhuyak Bay"
...,...,...
6399,16738,"Bentonville, AR: Bentonville Municipal Louise ..."
6400,16739,"Columbus, OH: Bolton Field"
6401,16740,"Delaware, OH: Delaware Municipal Jim Moore Field"
6402,16741,"Selmer, TN: Robert Sibley"


## Question 2: Summarize the data

print out summarizing data that helps you understand the data set.

In [139]:
print("{} entries in the data set".format(data.shape[0]))
print("{} ORIGIN AIRPORTS".format(np.unique(data["ORIGIN_AIRPORT_ID"]).size))
print("{} DEST AIRPORTS".format(np.unique(data["DEST_AIRPORT_ID"]).size))
print("{} DESCRIBED AIRPORTS".format(np.unique(lookup.index.tolist()).size))

## Solving large sparse linear systems

SciPy provides efficient routines for solving large sparse systems as for dense matrices. We will illustrate by calculating the page rank  for airports using data from the [Bureau of Transportation Statisitcs](http://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236). The [PageRank](https://en.wikipedia.org/wiki/PageRank) algorithm is used to rank web pages for search results, but it can be used to rank any node in a directed graph (here we have airports instead of web pages). PageRank is fundamentally about finding the steady state in a Markov chain and can be solved as a linear system.

The update at each time step for the page rank $PR$ of a page $p_i$ is 

![i0](https://wikimedia.org/api/rest_v1/media/math/render/svg/8a8c0a807f62331cc1740dd6c0f28ac1809926c7)

In the above equation, $B_u$ is the set of all nodes $v$ that link to $u$, where each $v$ node contributes its page rank divided by its number of outgoing links $L(v)$. So a node $v$ with a high page rank contributes a large value to a linked node $u$ if $v$ has relatively few other links. 

The figure shows a network with four nodes, all of which start with a page rank of $1/4$. The values on the edges shows how much of its page rank one nodes contributes to its linked nodes in the first step.

By letting the sum of all page ranks to be equal to one, we essentially have a probability distribution over the nodes of the graph. Since the state of the graph only depends on its previous state, we have a Markov chain. If we assume that every node can be reached from every other node, the system will have a steady state - which is what the PageRank algorithm seeks to find. To guard against case where a node has out-degree 0, we allow every node a small random chance of transitioning to any other node using a damping factor $d$. Then we solve  the linear system  to find the pagerank score $R$.


![i1](https://wikimedia.org/api/rest_v1/media/math/render/svg/6bb0f1469218a064274fd4691143e9ce64639dc2)

In matrix notation, this is

![i2](https://wikimedia.org/api/rest_v1/media/math/render/svg/96265e6c41318e793194287f36b5f929075bb876)

where

![i2.5](https://wikimedia.org/api/rest_v1/media/math/render/svg/3e82b446a376633a386b10668703a4547f167d1c)

At steady state,

![i3](https://wikimedia.org/api/rest_v1/media/math/render/svg/65d2fed50688deaca4640b117c88a9e7a3c2ef0d)

and we can rearrange terms to solve for $R$

![i4](https://wikimedia.org/api/rest_v1/media/math/render/svg/985f19f0c6b69d3a8afb5acc38339ebe4915baa7)



## Working with graphs 

We will use the Python package NetworkX for this example, but later in the week we will dig into this package.

In [114]:
import networkx as nx

## Question 3: create matrix that represents a graph

Use the following code to construct an [adjacency matrix](https://en.wikipedia.org/wiki/Adjacency_matrix), using the package NetworkX.

```python

g = nx.from_pandas_edgelist(data, source='ORIGIN_AIRPORT_ID', target='DEST_AIRPORT_ID')
airports = np.array(g.nodes())
adj_matrix = nx.to_scipy_sparse_matrix(g)
```

1. What kind of matrix are we working with?
2. Calculate the sparsity
3. Also stage the adjacency matrix for future use (save it to disk).

In [160]:
g = nx.from_pandas_edgelist(data, source='ORIGIN_AIRPORT_ID', target='DEST_AIRPORT_ID')
airports = np.array(g.nodes())
adj_matrix = nx.to_scipy_sparse_matrix(g)

print("object type", type(adj_matrix))
sparsity = 1.0 - (adj_matrix.count_nonzero() / adj_matrix.shape[0]**2)

## or because it is a fairly small matrix
A = adj_matrix.todense()
print('sparsity',round(1.0 - (np.count_nonzero(A) / A.size),3))

file_name = "airport_smatrix.npz"
sparse.save_npz(file_name, adj_matrix)
AA = sparse.load_npz(file_name)
print("object loaded from file", AA.shape)

OutDegreeView({10135: 3, 10397: 158, 11433: 99, 13930: 138, 10140: 21, 11298: 107, 14747: 71, 12478: 56, 13487: 108, 14869: 78, 12266: 109, 11292: 127, 14107: 78, 14771: 75, 12892: 73, 10821: 62, 11259: 52, 12191: 47, 12889: 74, 13198: 37, 13232: 62, 13796: 32, 14057: 43, 14679: 45, 10141: 1, 10146: 1, 10155: 1, 10157: 1, 10158: 6, 11697: 60, 13577: 5, 14027: 19, 13204: 68, 15304: 55, 14635: 35, 10165: 1, 10299: 22, 10170: 1, 10185: 3, 10208: 1, 10257: 12, 11057: 62, 11618: 82, 10279: 4, 12173: 28, 12758: 12, 10551: 1, 14709: 2, 10754: 3, 12523: 6, 10926: 2, 13873: 2, 13970: 2, 11630: 2, 13830: 18, 10333: 1, 10372: 7, 13303: 38, 14100: 38, 12953: 53, 13495: 36, 11066: 24, 13931: 10, 10423: 40, 11986: 13, 10721: 55, 11612: 3, 10529: 21, 10434: 4, 10693: 42, 12451: 17, 14122: 26, 14492: 31, 11252: 2, 13871: 17, 15323: 1, 15016: 45, 14307: 12, 11109: 8, 14683: 29, 11278: 44, 14908: 22, 11193: 24, 11973: 3, 14098: 1, 13244: 17, 14730: 15, 13360: 1, 10849: 10, 14893: 26, 11503: 9, 11540: 11

## Question 4: Construct the transition matrix

These are the necessary steps to construct a [transition matrix](https://en.wikipedia.org/wiki/Stochastic_matrix)

1. determine the out degree (the number of outward edges connected to a node)

The output will be a `np.array` with 294 elements each corresponding to the number of outbound flights

2. Create a diag_matrix using `scipy.sparse.diag` that uses `1/out_degrees`.  Then transform it to a csr matrix.

The matrix should be 294 x 294

3. Use matrix multiplication to multiply the diagonal matrix by the adjacency matrix.  Then transpose the result.

The matrix should be 294 x 294

In [161]:
## Answer
out_degrees = np.ravel(adj_matrix.sum(axis=1))
print(out_degrees.shape)
diag_matrix = sparse.diags(1 / out_degrees).tocsr()
print(diag_matrix.shape)
M = (diag_matrix @ adj_matrix).T
print(M.shape)

OutDegreeView({10135: 3, 10397: 158, 11433: 99, 13930: 138, 10140: 21, 11298: 107, 14747: 71, 12478: 56, 13487: 108, 14869: 78, 12266: 109, 11292: 127, 14107: 78, 14771: 75, 12892: 73, 10821: 62, 11259: 52, 12191: 47, 12889: 74, 13198: 37, 13232: 62, 13796: 32, 14057: 43, 14679: 45, 10141: 1, 10146: 1, 10155: 1, 10157: 1, 10158: 6, 11697: 60, 13577: 5, 14027: 19, 13204: 68, 15304: 55, 14635: 35, 10165: 1, 10299: 22, 10170: 1, 10185: 3, 10208: 1, 10257: 12, 11057: 62, 11618: 82, 10279: 4, 12173: 28, 12758: 12, 10551: 1, 14709: 2, 10754: 3, 12523: 6, 10926: 2, 13873: 2, 13970: 2, 11630: 2, 13830: 18, 10333: 1, 10372: 7, 13303: 38, 14100: 38, 12953: 53, 13495: 36, 11066: 24, 13931: 10, 10423: 40, 11986: 13, 10721: 55, 11612: 3, 10529: 21, 10434: 4, 10693: 42, 12451: 17, 14122: 26, 14492: 31, 11252: 2, 13871: 17, 15323: 1, 15016: 45, 14307: 12, 11109: 8, 14683: 29, 11278: 44, 14908: 22, 11193: 24, 11973: 3, 14098: 1, 13244: 17, 14730: 15, 13360: 1, 10849: 10, 14893: 26, 11503: 9, 11540: 11

## Question 5: Create the equation that we need to use to solve for steady states

use the following code to create the equation

```
n = len(airports) 
d = 0.85 
I = sparse.eye(n, format='csc')
A = I - d * M
b = (1-d) / n * np.ones(n) # so the sum of all page ranks is 1
```

For each line write a comment that explains the step in the process

In [30]:
## size of the system
n = len(airports) 

## damping factor (modulated the random chance of transitioning to any other node)
d = 0.85 

## component needed to represent first part of linear system
I = sparse.eye(n, format='csc')

## first part of linear system
A = I - d * M

## second part of linear system
b = (1-d) / n * np.ones(n) # so the sum of all page ranks is 1

print(A.shape)
print(b.shape)

## Question 6: Solve linear system

1. Use [spsolve](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.linalg.spsolve.html) to solve the linear system. Each value represents the page rank for a specific airport.

HINT: your result should sum to 1.0

2. use the sorted result to index back into your data to print Description, degree, and page rank.  Print the top 10.


In [31]:
from scipy.sparse.linalg import spsolve

In [32]:
## Answer
r =  spsolve(A, b)
r.sum()
idx = np.argsort(r)
top10 = idx[-10:][::-1]
bot10 = idx[:10]

In [32]:
df = lookup.loc[airports[top10]]
df['degree'] = out_degrees[top10]
df['pagerank']= r[top10]
df

In [32]:
df = lookup.loc[airports[bot10]]
df['degree'] = out_degrees[bot10]
df['pagerank']= r[bot10]
df

## Question 7:  Visualize the airport connections graph  (Extra Credit)

NetworkX has many [drawing capabilities](https://networkx.github.io/documentation/stable/reference/drawing.html).  

1. Go through [this tutorial](https://networkx.github.io/documentation/stable/tutorial.html) to familiarize yourself with how to work with graphs. Hint the variable, `g` should still be a NetworkX Graph object. 

2. Use the `Drawing Graphs` section to create a visualization of this network

3. Create a visualization that labels the top 5 airports by pagerank

In [33]:
## Answer


